# Typescript Snippets
---

## String replacement using TS 
helpful for managing type differences with PascalCase and snake_case 

In [ ]:
type StringReplace< 
  TString extends string, 
  TToReplace extends string, 
  TReplacement extends string 
> = TString extends `${infer TPrefix}${TToReplace}${infer TSuffix}` 
  ? `${TPrefix}${TReplacement}${StringReplace<
    TSuffix, 
    TToReplace, 
    TReplacement
    >}`
  : TString;

type Result = StringReplace<"send_email_new", "_", "-">;

type CheckConditional = "send_email" extends `${infer TPrefix}${"_"}${infer TSuffix}`
  ? [TPrefix, TSuffix]
  : false;

type CheckReplacement = 
  "cool_something" extends `${infer TPrefix}${"_"}${infer TSuffix}`
    ? `${TPrefix}${"-"}${TSuffix}`
    : "cool_something"


---
## Optional Properties - Alternative strategy 
### use Omit`<T,key>` instead of `?` 
Avoid making a property optional to support two separate scenarios; doing so reduces type safety

In [ ]:
type user = {
	// making id optinal since unsaved users don't have an id yet 👎
	id?: number;
	name: string;
	email: string;
};

// instead, declare separate types. Then each scenario is fully type safe 👍
type User = {
	id: number;
	name: string;
	email: string;
};

// separate type for unsaved users
// deriving from the User type via Omit to keep types clean and lean
type UnsavedUser = Omit<User, 'id'>;

---
## Index Access Types
### Deep access of arrays and objects

In [ ]:
interface ColorVariants {
    primary: 'blue'
    secondary: 'red'
    tertiary: 'green'
}

// use index access type 👍
type PrimaryColor = ColorVariants['primary']

// with unions
type NonPrimaryColor = ColorVariants['secondary' | 'tertiary']
// with bigger unions
type EveryColor = ColorVariants[keyof ColorVariants]

// array of letters
type Letters = ['a', 'b', 'c']
// just 'a' or 'b'
type AorB = Letters [0 | 1];
// union of all array elements
type Letter = Letters[number]

// here's the cool one 🚀
interface UserRoleConfig {
    user: ['view', 'create', 'update'];
    superAdmin: ['view', 'create', 'update', 'delete'];
}

type Role = UserRoleConfig[keyof UserRoleConfig][number]

If you're getting the TypeScript error

'...expression of type string cannot be used to index...'

then simply specify that the 'expression of type string' is a key of the type of that object. For example,

```
const someObj:ObjectType = data;
const field = 'username';
```

// This gives an error

```const temp = someObj[field];```

// Solution 1: When the type of the object is known

```const temp = someObj[field as keyof ObjectType]```

// Solution 2: When the type of the object is not known

```const temp = someObj[field as keyof typeof someObj]```

---
## Generics in React Components
leverage generics to create flexible and dynamic components

In [ ]:
import React from 'react'

//* goal is to pass any items, and have the id of that 
//* item propagate through to 2nd (render) function

interface TableProps<TItem> {
    // items: {id: string}[]
    items: TItem[]; // replaced with generic TItem
    // renderItem: (item: {id: string}) => React.ReactNode
    renderItem: (item: TItem) => React.ReactNode // replace with generic TItem
}

// need function syntax not arrow to avoid error
export function Table<TItem>(props: TableProps<TItem>) {
    return null;
}

const Component = () => {
    return (
        // <Table<{id: number}> // can manually specify the generic
        <Table
            items={[{ id: "1", name: 'Jim'}]}
            renderItem={(item) => (<div>{item.id}</div>)}
        ></Table>
    )
}

### Passing generics to types
10 examples from https://www.youtube.com/embed/dLPgQRbVquo

In [ ]:
// pass types to other types
type MyGenericType<TData> = {
  data: TData;
}

// type Example inherits type from the generic
type Example = MyGenericType<{
  firstName: string;
}>

type Example2 = MyGenericType<number>

export {};

### Passing types to functions

In [ ]:
// generic functions are funcs with a type helper mapped over the top, 
// which mapes the type to the return value

const fetchSomething = <TData>(url:string): Promise<TData> => {
  return fetch(url).then((res) => res.json());
}

fetchSomething<{firstName: string; lastName: string}>(
  "/api/endpoint"
).then((res) => {
  console.log(res)
})

### Passing types to Sets

In [ ]:
//* pass type params to other JS stuff, like Sets and Maps

const set = new Set<number>();

set.add(1)

set.add("some string") //! errors due to generic type requiring number type


### Inferring types passed to functions

In [ ]:
//* you don't always have to pass types to a generic function!

const addIdToObject = <TObject>(obj: TObject) => {
  return {
    ...obj,
    id: "123"
  }
}
// types are inferred from func args
const result = addIdToObject({
  firstName: "Jim",
  lastName: "Deola"
});

### Generic Constraints ⭐️

In [ ]:
//* ReturnType's type def expects a function
// type GetPromiseReturnType<T> = Awaited<ReturnType<T>>;

//* so, we add 'extends' to our generic type as a constraint as well 
type GetPromiseReturnType<T extends (...args: any) => any> = Awaited<ReturnType<T>>;
//* Awaited used for async stuff like when using 'await'

type MyResult = GetPromiseReturnType<
  () => Promise<{
    firstName: string;
    lastname: string;
  }>
>

type ErrorLine = GetPromiseReturnType<string> //! errors due to constraint

### Advanced Generic Constraints ⭐️

In [ ]:
//* sometimes you need to contrain the generic that gets passed in

//* we pass an object to function and returns an object with the key of the Tobj and value
//* Record is type helper that gives you an object with dynamic keys (as string, here) AND specified value
const getKeyWithHighestValue = <TObj extends Record<string, number>>(
  obj: TObj
): {
  key: keyof TObj;
  value: number;
} => {
  const keys = Object.keys(obj) as Array<keyof TObj>;

  let highestKey: keyof TObj = keys[0];
  let highestValue = obj[highestKey];

  for (const key of keys) {
    if (obj[key] > highestValue) {
      highestKey = key;
      highestValue = obj[key]
    }
  }
    return {
      key: highestKey,
      value: highestValue
    }
};

const finalResult = getKeyWithHighestValue({
  a: 1,
  b: 2,
  c: 3
});

const key = finalResult.key;
const value = finalResult.value;

### Overriding Generics

In [ ]:
// use an assertion to override the expected type from the generic

const typedObjectKeys = <TObj extends {}>(
  obj: TObj
): Array<keyof TObj> => { // ⬅️ can remove Array<keyof TObj>
  // return Object.keys(obj) //! this errors bc TS can get confused with complex generics
  return Object.keys(obj) as Array<keyof TObj>
}

const resultObj = typedObjectKeys({
  name: "Jim",
  age: 33
});

### Multiple Generics ⭐️

In [ ]:
//* we need to constrain TKey but still infer it
const getValue1 = <TObj, TKey extends keyof TObj>(obj: TObj, key: TKey) => {
  if (key === "bad") {
    throw Error("Don't access the bad key!")
  }
  return obj[key]
}

const resultValue = getValue1({
  a: 1,
  b: "some string"
},
"b"
);

console.log(resultValue);

### Default Generics

In [ ]:
//* we add a default type to generic type T, 
//* since there are no function arguments to infer from

const createSet = <T = string>() => {
  return new Set<T>();
}

const numberSet = createSet<number>();
const stringSet = createSet<string>();

const otherStringSet =  createSet();

### use generics to infer from 3rd party libs

In [ ]:
import {z} from "zod";
//* using zod to set schema and validate types, we can provide safety at runtime

const safeZodFetch = <TData>(
  url: string,
  schema: z.Schema<TData>    // z.Schema accepts type args
): Promise<TData> => {
  return fetch(url)
    .then((res) => res.json())
    .then(res => {
    return schema.parse(res);
  })
};

const safeZodFetchResult = safeZodFetch //<
// {
//   firstName: string; //* now we can remove these type args
//   lastName: string;  //* and result will still be inferred safely on type and runtime levels
// }
//>
(
  "/api/endpoint",
  z.object({
    firstName: z.string(),
    lastName: z.string()
  })
).then((res) => {
  console.log(res);
})

//* Note: by grouping complex typescript 'stuff' in one place, 
//* we can often avoid needing it in our pure js functions that we use with it.   

---
## Turning a module into a type

``` export type Action = "ADD_TODO" | "REMOVE_TODO" | "EDIT_TODO" ```

This is a common pattern in older Redux applications. Now, we've actually got a constants.ts file which has all of the elements of the union. They're even inferred by TypeScript

The exports are not actually inferred as strings. They're inferred as the literals "ADD_TODO", "REMOVE_TODO", and "EDIT_TODO".

/constants.ts
```
export const ADD_TODO = "ADD_TODO"
export const REMOVE_TODO = "REMOVE_TODO"
export const EDIT_TODO = "EDIT_TODO"
```

So there must be a way that we can extract this information and create the union type dynamically. Because otherwise, whenever we add a new element in constants.ts, we've also got to add it to the new place. It's just not very DRY.

In [ ]:
export type Action1 = "ADD_TODO" | "REMOVE_TODO" | "EDIT_TODO"

export type ActionModule = typeof import("./constants") 

// create a union type out of the keys of ActionModule
export type Action2 = ActionModule[keyof ActionModule]

This takes the exported keys of the ActionModule, and it sort of iterates over them. So we end up with ADD_TODO, REMOVE_TODO, and EDIT_TODO which stays in sync with our actual code.

---
### Fix Object Keys with `infer` and template literals


In [ ]:
// how to remove unnecessary 'maps' from object keys
interface ApiData {
    'maps:longitude': string;
    'maps:latitude': string;
}

type RemoveMapsFomObj<T> = {
    [K in keyof T as RemoveMaps<K>]: T[K];
};

type DesiredShape = RemoveMapsFomObj<ApiData>
// yields type DesiredShape = { longitude: string, latitude: string }

type RemoveMaps<T> = T extends `maps:${infer U}` ? U : T

---
## Typesafe React Context

In [ ]:
import react
import { createContext, useContext} from "react"

export interface UserContextType {
    name: string;
    age: number;
}

export const userContext = createContext<UserContextType | null>(null);

export const useUserContext = () => {
    const context = useContext(userContext);
    // handle errors to 
    if (!context) {
        throw new Error("useUserContext msut be used within a UserProvider!")
    }
}


---
## Intersection Observer React Hook

read more here
https://betterprogramming.pub/react-useinview-hook-intersection-observer-animations-and-multiple-refs-73c68a33b5b1

In [ ]:

import React, { useState, useEffect } from 'react';

export function useInView(refs: React.RefObject<HTMLElement>[]) {
  const [elements, setElements] = useState<{
    [key: string]: { isInView: boolean };
  }>({});

  useEffect(() => {
    const observerCallback = (entries: IntersectionObserverEntry[]) => {
      entries.forEach((entry) => {
        const name = entry.target.getAttribute('data-myProperty');
        if (!name) {
          console.warn(
            'Encountered entry with no name. You should add data-myProperty to every element passed to the isInView hook.'
          );
        } else {
          if (entry.isIntersecting) {
            setElements((prev) => {
              return {
                ...prev,
                [name]: {
                  isInView: true,
                },
              };
            });
          } else {
            setElements((prev) => ({
              ...prev,
              [name]: {
                isInView: false,
              },
            }));
          }
        }
      });
    };

    const observer = new IntersectionObserver(observerCallback);

    refs.forEach((ref) => {
      if (ref.current) {
        observer.observe(ref.current);
      }
    });

    return () => {
      observer.disconnect();
    };
  }, []);

  return elements;
}

---
### Type-Safe fetching with Zod
https://twitter.com/mattpocockuk/status/1610948469488771074?s=12&t=Ua6guZt5woaom80XYXfdxQ

In [ ]:
// client.ts

// import type { RouteMap } from "./server";

export const fetchFromBackend = <
  TRoute extends keyof RouteMap
>(
  route: TRoute,
  input: Parameters<RouteMap[TRoute]>[0]
) => {
  return fetch(route, {
    method: "POST",
    body: JSON.stringify(input),
  }).then((res) => res.json()) as ReturnType<
    RouteMap[TRoute]
  >;
};

// makeTypeSafeApiCall.ts

import { z } from "zod";

type MaybePromise<T> = T | Promise<T>;

export const makeTypeSafeApiCall = <TInput, TOutput>(
  inputSchema: z.Schema<TInput>,
  outputSchema: z.Schema<TOutput>,
  handler: (input: TInput) => MaybePromise<TOutput>
): ((input: TInput) => MaybePromise<TOutput>) => {
  return async (input: unknown) => {
    const result = await handler(inputSchema.parse(input));
    return outputSchema.parse(result);
  };
};

// server.ts

// import { z } from "zod";
// import { makeTypeSafeApiCall } from "./makeTypeSafeApiCall";

export type RouteMap = {
  "/user/create": typeof createUser;
};

export const createUser = makeTypeSafeApiCall(
  z.object({
    email: z.string(),
  }),
  z.object({
    id: z.string(),
    email: z.string(),
  }),
  async ({ email }) => {
    return {
      id: "123",
      email,
    };
  }
);